<a href="https://colab.research.google.com/github/leonardo3108/ia025a/blob/main/Exercicios/Aula4-Regressao_Softmax_MNIST_SGD_minibatches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão Softmax com dados do MNIST utilizando gradiente descendente estocástico por minibatches

Este exercicío consiste em treinar um modelo de uma única camada linear no MNIST **sem** usar as seguintes funções do pytorch:

- torch.nn.Linear
- torch.nn.CrossEntropyLoss
- torch.nn.NLLLoss
- torch.nn.LogSoftmax
- torch.optim.SGD
- torch.utils.data.Dataloader

## Importação das bibliotecas

In [90]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torchvision
from torchvision.datasets import MNIST

## Fixando as seeds

In [91]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Dataset e dataloader

### Definição do tamanho do minibatch

In [92]:
batch_size = 50

### Carregamento, criação dataset e do dataloader

In [93]:
dataset_dir = '../data/'

dataset_train_full = MNIST(dataset_dir, train=True, download=True,
                           transform=torchvision.transforms.ToTensor())
print(dataset_train_full.data.shape)
print(dataset_train_full.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


### Usando apenas 1000 amostras do MNIST

Neste exercício utilizaremos 1000 amostras de treinamento.

In [94]:
indices = torch.randperm(len(dataset_train_full))[:1000]
dataset_train = torch.utils.data.Subset(dataset_train_full, indices)

In [95]:
# Escreva aqui o equivalente do código abaixo:
# loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False)

from math import ceil

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True):
        self.batch_size = batch_size
        if shuffle:
            self.dataset = dataset[torch.randperm(len(dataset))]
        else:
            self.dataset = dataset
        self.batch_number = ceil(len(self.dataset) / self.batch_size)

    def __len__(self):
        return self.batch_number
    
    def __iter__(self):
        self.dataset_index = 0
        return self

    def __next__(self):
        batch_size = self.batch_size
        if batch_size + self.dataset_index > len(self.dataset):
            batch_size = len(self.dataset) - self.dataset_index
            if self.dataset_index >= len(self.dataset):
                raise StopIteration
        for batch_index in range(batch_size):
            data, target = self.dataset[batch_index + self.dataset_index]
            if batch_index == 0:
                batch_data = data
                batch_targets = torch.tensor([target])
            else:
                batch_data = torch.cat((batch_data, data))
                batch_targets = torch.cat((batch_targets, torch.tensor([target])))
        self.dataset_index += batch_size
        return batch_data, batch_targets

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)

In [96]:
print('Número de minibatches de trenamento:', len(loader_train))

x_train, y_train = next(iter(loader_train))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 20

Dimensões dos dados de um minibatch: torch.Size([50, 28, 28])
Valores mínimo e máximo dos pixels:  tensor(0.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


## Modelo

In [97]:
# Escreva aqui o codigo para criar um modelo cujo o equivalente é: 
# model = torch.nn.Linear(28*28, 10)
# model.load_state_dict(dict(weight=torch.zeros(model.weight.shape), bias=torch.zeros(model.bias.shape)))


import torch
from torch.nn import Parameter, Module

class Model(Module):
    def __init__(self):
        super(Model, self).__init__()
        self.weight = Parameter(torch.randn(28*28, 10, requires_grad=True))
        self.bias = Parameter(torch.randn(10, requires_grad=True))

    def print_parameters(self):
        print('Weights -', self.weight, self.weight.shape)
        print('Biases -', self.bias, self.bias.shape)
        print()
    
    def forward(self, x):
        return torch.matmul(x, self.weight) + self.bias

In [98]:
model = Model()
#model.load_state_dict(dict(weight=torch.zeros(model.weight.shape), bias=torch.zeros(model.bias.shape)))
model.print_parameters()
h = model.forward(torch.ones(28*28))

Weights - Parameter containing:
tensor([[-0.7044, -1.9987,  1.3757,  ...,  1.1866,  1.0854,  0.2307],
        [-0.0964, -0.7126,  1.3280,  ...,  1.0290,  0.3860,  0.2508],
        [ 0.0476,  1.3403, -1.1607,  ..., -1.6446, -0.1817,  0.9301],
        ...,
        [ 0.2880,  1.4009,  0.0790,  ...,  0.1595, -0.2142,  0.9612],
        [ 1.3316,  0.0113, -0.4440,  ..., -1.4287, -0.4731, -1.5490],
        [ 2.2793,  0.2803,  0.3863,  ...,  1.4419, -0.2467,  1.3578]],
       requires_grad=True) torch.Size([784, 10])
Biases - Parameter containing:
tensor([ 0.6594, -1.0633,  0.1111, -0.4109,  0.6796,  0.0609, -0.1880, -0.2454,
         0.0845, -1.3243], requires_grad=True) torch.Size([10])



## Treinamento

### Inicialização dos parâmetros

In [99]:
n_epochs = 50
lr = 0.1

## Definição da Loss



In [186]:
# Escreva aqui o equivalente de:
# criterion = torch.nn.CrossEntropyLoss() = LogSoftmax o NLLLoss.
from torch import Tensor


class CrossEntropyLoss():
    def __call__(self, logits: 'Tensor', target: 'Tensor'):
        #to do: one hot de target
        softmax = logits.exp() / logits.exp().sum()
        logSoftmax = softmax.log()
        loss = torch.sum(-target * logSoftmax, dim=-1)
        mean_loss = loss.mean(dim=0)
        #print('logits', logits)
        #print('softmax', softmax, 'sum', softmax.sum())
        #print('logSoftmax', logSoftmax, 'mul', logSoftmax.prod())
        #print('Negative Log-Likelihood', loss)
        return loss

criterion = CrossEntropyLoss()

criterion(h, 4)

tensor(1501.5013, grad_fn=<SumBackward1>)

# Definição do Optimizer

In [175]:
# Escreva aqui o equivalente de:
# optimizer = torch.optim.SGD(model.parameters(), lr)

class SGD():
    def __init__(self, parameters, learning_rate: float):
        self.parameters = parameters
        self.learning_rate = learning_rate

    def step(self):
        for param in self.parameters:
            param.data -= self.learning_rate * param.grad

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad *= 0        

optimizer = SGD(model.parameters(), lr)

### Laço de treinamento dos parâmetros

In [184]:
epochs = []
loss_history = []
loss_epoch_end = []
total_trained_samples = 0
for i in range(n_epochs):
    # Substitua aqui o loader_train de acordo com sua implementação do dataloader.
    for x_train, y_train in loader_train:
        # Transforma a entrada para uma dimensão
        inputs = x_train.view(-1, 28 * 28)
        # predict da rede
        outputs = model(inputs)
        print(inputs.shape, outputs.shape, y_train.shape)
        print(y_train)

        # calcula a perda
        loss = criterion(outputs, y_train)

        # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
        # Escreva aqui o código cujo o resultado é equivalente às 3 linhas abaixo:
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(dataset_train))
        loss_history.append(loss.item())

    loss_epoch_end.append(loss.item())
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss: {loss.item()}')


torch.Size([50, 784]) torch.Size([50, 10]) torch.Size([50])
tensor([1, 3, 1, 8, 3, 4, 0, 0, 9, 5, 3, 3, 6, 4, 0, 8, 0, 8, 9, 6, 8, 1, 6, 7,
        9, 8, 2, 0, 8, 0, 7, 2, 2, 4, 0, 1, 3, 3, 5, 3, 3, 8, 0, 1, 3, 7, 8, 3,
        3, 9])


RuntimeError: ignored

### Visualizando gráfico de perda durante o treinamento

In [ ]:
plt.plot(epochs, loss_history)
plt.xlabel('época')

### Visualização usual da perda, somente no final de cada minibatch

In [ ]:
n_batches_train = len(loader_train)
plt.plot(epochs[::n_batches_train], loss_history[::n_batches_train])
plt.xlabel('época')

In [ ]:
# Assert do histórico de losses
target_loss_epoch_end = np.array([
    1.1979684829711914,
    0.867622971534729,
    0.7226786613464355,
    0.6381281018257141,
    0.5809749960899353,
    0.5387411713600159,
    0.5056464076042175,
    0.4786270558834076,
    0.4558936357498169,
    0.4363219141960144,
    0.4191650450229645,
    0.4039044976234436,
    0.3901679515838623,
    0.3776799440383911,
    0.3662314713001251,
    0.35566139221191406,
    0.34584277868270874,
    0.33667415380477905,
    0.32807353138923645,
    0.31997355818748474,
    0.312318354845047,
    0.3050611615180969,
    0.29816246032714844,
    0.29158851504325867,
    0.28531041741371155,
    0.2793029546737671,
    0.273544579744339,
    0.2680158317089081,
    0.26270008087158203,
    0.2575823664665222,
    0.25264936685562134,
    0.24788929522037506,
    0.24329163134098053,
    0.23884665966033936,
    0.23454584181308746,
    0.23038141429424286,
    0.22634628415107727,
    0.22243399918079376,
    0.2186385989189148,
    0.21495483815670013,
    0.21137762069702148,
    0.20790249109268188,
    0.20452524721622467,
    0.20124195516109467,
    0.19804897904396057,
    0.1949428766965866,
    0.19192075729370117,
    0.188979372382164,
    0.18611609935760498,
    0.1833282858133316])

assert np.allclose(np.array(loss_epoch_end), target_loss_epoch_end, atol=1e-6)

## Exercício 

Escreva um código que responda às seguintes perguntas:

Qual é a amostra classificada corretamente, com maior probabilidade?

Qual é a amostra classificada erradamente, com maior probabilidade?

Qual é a amostra classificada corretamente, com menor probabilidade?

Qual é a amostra classificada erradamente, com menor probabilidade?

In [ ]:
# Escreva o código aqui:

## Exercício Bonus

Implemente um dataloader que aceite como parâmetro de entrada a distribuição probabilidade das classes que deverão compor um batch.
Por exemplo, se a distribuição de probabilidade passada como entrada for:

`[0.01, 0.01, 0.72, 0.2, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]`

Em média, 72% dos exemplos do batch deverão ser da classe 2, 20% deverão ser da classe 3, e os demais deverão ser das outras classes.

Mostre também que sua implementação está correta.
